# Gemini File Search Tutorial

This notebook demonstrates how to use Google's Gemini File Search API to create a Retrieval-Augmented Generation (RAG) system.

In [ ]:
import os
from google import genai
from dotenv import load_dotenv

load_dotenv()

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

## 1. Uploading Documents
First, we upload the documents we want to search. Gemini handles the chunking and embedding automatically.

In [ ]:
# Upload a file
sample_file = client.files.upload(
    file='businesszerotoone.pdf',
    config={'name': 'business_sample'}
)
print(f"Uploaded: {sample_file.name}")

## 2. Creating a File Search Store
A store acts as a container for your documents (like a vector database).

In [ ]:
store = client.file_search_stores.create(
    config={'display_name': 'Business Knowledge Base'}
)
print(f"Created Store: {store.name}")

## 3. Importing Files into the Store
We link the uploaded file to the store.

In [ ]:
import_operation = client.file_search_stores.import_file(
    file_search_store_name=store.name,
    file_name=sample_file.name
)

# Wait for completion (optional, but good practice)
import time
while not import_operation.done:
    time.sleep(1)
    import_operation = client.operations.get(import_operation)
print("Import completed")

## 4. Querying with Custom Chunking
You can customize how Gemini chunks the documents for better retrieval.

In [ ]:
# Example of querying (the actual chunking config is set during upload/ingestion in some versions, 
# or handled by the model's tool configuration)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the main concept of Zero to One?",
    config={
        'tools': [{
            'file_search': {
                'file_search_store_names': [store.name]
            }
        }]
    }
)

print(response.text)